In [1]:
from __future__ import print_function
from __future__ import division
from future import standard_library
import time
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from scipy.cluster import *
import word2vec

def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record
        
model = word2vec.load('/Users/ivan/Downloads/182/withOutTypeModel.txt')

In [ ]:
def process(pipeline, text='некий текст', keep_pos=True, keep_punct=False):
    wordInfo = pipeline.process(text).split('\n')[4].split('\t')
    if (wordInfo[3] == 'NUM'):
        return ('_NUM_' +'x' * len(word[2]))
    else:
        return wordInfo[2]

def tag_ud(text='некий текст'):
    model = Model.load('udpipe_syntagrus.model')
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

    #print('Processing input...', file=sys.stderr)
    return process(process_pipeline, text=text)
#print (process(Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu'),"ксюша"))
#https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb

In [19]:
modelForInfinitive1 = Model.load('udpipe_syntagrus.model')

In [152]:
%%time
class wordsClass (object):
    model
    vectorOfStr = torch.zeros(2000)
    centroids = torch.zeros(2000,300)
    ClasterOfWord = {}
    nomberKnownOfWords = 0
    modelForInfinitive = ''

    def loadCentroidsFromTXT (self, source):
        self.centroids = torch.from_numpy(np.loadtxt(source, delimiter=' ', dtype=float))
        self.centroids = torch.reshape(self.centroids, (2000, 300)).cuda()
        
    def saveCentroidsToTXT (self, source):
        f = open (source, 'w')
        for i in range (2000):
            for j in range (300):
                f.write(str(self.centroids[i][j]) + " ")
        f.close()
        
    def saveDict (self, source):
        f = open(source, 'w')
        for k, v in self.ClasterOfWord.items():
            f.write(str(k) + " " + str(v) + "\n")
        f.close()
            
    def loadDict (self, source):
        f = open(source, 'r')
        for strings in f:
            strings = strings.split(' ')
            self.ClasterOfWord.update({strings[0]: int(strings[1])})
        f.close()
        
    def generateCentroids (self, nomberOfClasters, NomberOfIter):
        #scipy.cluster.vq.kmeans2(model.vectors, 2000, iter=100)
        self.centroids = vq.kmeans(self.model.vectors, nomberOfClasters, iter=NomberOfIter)[0]
        
    def generateDict (self, source): #источник = txt для model
        f = open (source, 'r')
        n=0
        words = []
        for line in f:
            line = line.split()
            words.append(line[0])
            n = n+1
        print (words[1])
        f.close()
        for word in log_progress(words, every=100):
            distToCentroids = torch.sum(((self.centroids - self.model[word])**2),1)
            relationID = int(torch.argmin(distToCentroids, 0))
            ClasterOfWord.update({word: relationID})
    
    def distanceBetween2words (self, word1, word2):
        word1 = word1.lower()
        word2 = word2.lower()
        try:
            len = model.distance(tag_ud(text=word1), tag_ud(text=word2))
        except Exception:
            try:
                len =  model.distance(tag_ud(text=word1.capitalize()), tag_ud(text=word2))
            except Exception:
                try:
                    len =  model.distance(tag_ud(text=word1), tag_ud(text=word2.capitalize()))
                except Exception:
                    len = model.distance(tag_ud(text=word1.capitalize()), tag_ud(text=word2.capitalize()))
        return len
    
    def coordinate (self, word):
        word = word.lower()
        try:
            coord = self.model[tag_ud(text=word)]
        except Exception:
            try:
                coord =  self.model[tag_ud(text=word.capitalize())]
            except Exception:
                coord = "error"
        return torch.from_numpy (coord)
    
    def getKey(self, value):
        for k, v in self.ClasterOfWord.items():
            if int(v) == value:
                print (k)
        return k
    
    def addNewWordInClaster (self, newWord, baseWord):
        self.ClasterOfWord.update({newWord: self.ClasterOfWord[baseWord]})
    
    def tryToUpgradeSelfWord2vec (self, word):
        try:
            #print (word)
            relationID = self.ClasterOfWord[word.lower()]
            self.vectorOfStr[relationID] = self.vectorOfStr[relationID] + 1
            self.nomberKnownOfWords = self.nomberKnownOfWords + 1
            #print (relationID)
        except Exception:
            if not re.search(r'[\W]', word):
                #print(self.wordToInf(text=word))
                relationID = self.ClasterOfWord[self.wordToInf(text=word)]
                self.vectorOfStr[relationID] = self.vectorOfStr[relationID] + 1
                self.nomberKnownOfWords = self.nomberKnownOfWords + 1
                #print (relationID)
            else: 
                raise
                
            
    def text2Vec (self, text):
        self.vectorOfStr = torch.zeros(2000)
        words = re.findall(r'[0-9A-Za-zА-Яа-я-.]+', text)
        for word in words:
            try :
                self.tryToUpgradeSelfWord2vec(word)
            except Exception:
                for singleWord in re.findall(r"[\w']+", word):
                    try: 
                        self.tryToUpgradeSelfWord2vec(singleWord)
                    except Exception:
                        print ("the word is new: " + singleWord)
        self.vectorOfStr = self.vectorOfStr / self.nomberKnownOfWords    
        return self.vectorOfStr
    
    def wordToInf(self, text):
        process_pipeline = Pipeline(self.modelForInfinitive, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        wordInfo = process_pipeline.process(text).split('\n')[4].split('\t')
        if (wordInfo[3] == 'NUM'):
            return ('_NUM_' + ('x' * len(wordInfo[2])))
        else:
            return wordInfo[2]
        
#проработать встречи ::
#реализовать работу с реляционной БД (?):
#(текст->оценка принадлежности вектор->оценка принадлежности)

wordsClass = wordsClass()
wordsClass.modelForInfinitive = modelForInfinitive1
wordsClass.loadDict('correct_dictionary.txt')
#wordsClass.loadDict('correct_dictionary.txt')
wordsClass.text2Vec("12 3 444 Китайский интернет-гигант Alibaba меняет бизнес-модель торговой площадки AliExpress, дав возможность продавцам из России, а также Турции, Италии и Испании продавать на ней свои товары. Об этом сообщает «Интерфакс».")
#wordsClass.text2Vec("12 3 444 вставал")
#wordsClass.ClasterOfWord['Alibaba']
#wordsClass.ClasterOfWord.update({'AliExpress': int(1829)})
#wordsClass.saveDict('dict words has clasters.txt')

the word is new: а
the word is new: и
the word is new: Об
CPU times: user 209 ms, sys: 11.2 ms, total: 221 ms
Wall time: 220 ms
